# Data-Centric NLP 대회: 주제 분류 프로젝트

## Load Libraries

In [1]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-9sq5xqax
  Running command git clone --filter=blob:none --quiet 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-9sq5xqax
  Resolved https://****@github.com/SKTBrain/KoBERT.git to commit 47a69af87928fc24e20f571fe10c3cc9dd9af9a3
  Preparing metadata (setup.py) ... done


In [2]:
import os
import numpy as np
import pandas as pd

import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp

from tqdm.notebook import tqdm

In [3]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [4]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [5]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score, f1_score, precision_score, recall_score

## Set Hyperparameters

In [6]:
DEVICE = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
DEVICE

/opt/conda/lib/python3.8/site-packages/torch/cuda/__init__.py:52: UserWarning: CUDA initialization: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx (Triggered internally at  /opt/conda/conda-bld/pytorch_1607370172916/work/c10/cuda/CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


device(type='cpu')

In [7]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, '../data')
OUTPUT_DIR = os.path.join(BASE_DIR, '../output')
SEED = 42

In [8]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

## Load Tokenizer and Model

In [9]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /opt/ml/code/.cache/kobert_v1.zip
using cached model. /opt/ml/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /opt/ml/code/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


## Define Dataset

In [11]:
data = pd.read_csv(os.path.join(DATA_DIR, 'train.csv'))
dataset_train, dataset_eval = train_test_split(data, train_size=0.7, random_state=SEED)

In [12]:
dataset_train.head()

,ID,input_text,label_text,target,predefined_news_category,annotations,url,date
4824,ynat-v1_train_04173,특징주 증권주 증시 상승 흐름 재개에 동반 강세,경제,1,경제,"{'annotators': ['12', '09', '11'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.03.12. 오전 9:27
31059,ynat-v1_train_27054,김현 1골 1도움 아산 부산에 역전승…K리그2 선두 ...,스포츠,6,스포츠,"{'annotators': ['12', '14', '09'], 'annotation...",https://sports.news.naver.com/news.nhn?oid=001...,2018.09.22 21:00
32729,ynat-v1_train_28483,네이버 D2 스타트업 팩토리 방문한 방송통신위원장,경제,1,IT과학,"{'annotators': ['02', '12', '11'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2018.07.20. 오후 6:18
40357,ynat-v1_train_35099,이란 美와 수감자 더 교환 가능…미국에 공 넘어가,세계,4,세계,"{'annotators': ['16', '08', '07'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.12.09. 오후 11:36
23873,ynat-v1_train_20800,美정부 연일 中화웨이 공격…5G장비 구축 LGU 곤혹,세계,4,IT과학,"{'annotators': ['02', '10', '14'], 'annotation...",https://news.naver.com/main/read.nhn?mode=LS2D...,2019.02.13. 오후 5:24


In [13]:
class BERTDataset(Dataset):
    def __init__(self, dataset, bert_tokenizer, max_len, pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)
        texts = dataset['input_text'].tolist()
        targets = dataset['target'].tolist()

        self.sentences = [transform([i]) for i in texts]
        self.labels = [np.int32(i) for i in targets]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [14]:
data_train = BERTDataset(dataset_train, tok, max_len, True, False)
data_eval = BERTDataset(dataset_eval, tok, max_len, True, False)

In [15]:
train_dataloader = DataLoader(data_train, batch_size=batch_size)
eval_dataloader = DataLoader(data_eval, batch_size=batch_size)

## Define Model

In [16]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=7,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [17]:
model = BERTClassifier(bertmodel, dr_rate=0.5).to(DEVICE)

## Define Optimizer and Scheduler

In [18]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [19]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [20]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [21]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

## Define Metric

In [22]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

## Train Model

In [23]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(DEVICE)
        segment_ids = segment_ids.long().to(DEVICE)
        valid_length= valid_length
        label = label.long().to(DEVICE)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
        token_ids = token_ids.long().to(DEVICE)
        segment_ids = segment_ids.long().to(DEVICE)
        valid_length= valid_length
        label = label.long().to(DEVICE)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

epoch 1 batch id 1 loss 1.987795114517212 train acc 0.15625



KeyboardInterrupt: 

## Evaluate Model

In [ ]:
dataset_eval = pd.read_csv(os.path.join(DATA_DIR, 'test.csv'))
dataset_eval['target'] = [0]*len(dataset_eval)
data_eval = BERTDataset(dataset_eval, tok, max_len, True, False)
eval_dataloader = DataLoader(data_eval, batch_size=batch_size, shuffle=False)

In [ ]:
preds = []
model.eval()
for batch_id, (token_ids, valid_length, segment_ids, _) in tqdm(enumerate(eval_dataloader), total=len(eval_dataloader)):
    token_ids = token_ids.long().to(DEVICE)
    segment_ids = segment_ids.long().to(DEVICE)
    valid_length= valid_length
    out = model(token_ids, valid_length, segment_ids)
    max_vals, max_indices = torch.max(out, 1)
    preds.extend(list(max_indices))

print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))
preds = [int(p) for p in preds]

In [ ]:
dataset_eval['target'] = preds
dataset_eval.to_csv(os.path.join(BASE_DIR, 'output.csv'), index=False)